In [33]:
from transformers import AutoModelForImageClassification, AutoImageProcessor, pipeline
from PIL import Image
from datasets import load_dataset
import os, torch

In [34]:
ds = load_dataset("imagefolder", data_dir="/home/dxd_jy/joel/Capstone/Training_Testing/Test/Test_Images/Test2")['train']

model_checkpoint_1 = "google/vit-base-patch16-224"
model_checkpoint_2 = "microsoft/beit-base-patch16-384"

inf_image_processor_1 = AutoImageProcessor.from_pretrained(model_checkpoint_1)
model_1 = AutoModelForImageClassification.from_pretrained("/home/dxd_jy/joel/Capstone/Model/best-vit-base-patch16-224-10L_20E_8B_5e-05_0.3")
pipe1 = pipeline("image-classification", model_1, image_processor=inf_image_processor_1)

inf_image_processor_2 = AutoImageProcessor.from_pretrained(model_checkpoint_2)
model_2 = AutoModelForImageClassification.from_pretrained("/home/dxd_jy/joel/Capstone/Training_Testing/Train/beit-base-patch16-384-10L_10E_8B_5e-05_0.3")
pipe2 = pipeline("image-classification", model_2, image_processor=inf_image_processor_2)


Resolving data files:   0%|          | 0/8936 [00:00<?, ?it/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [35]:
weight1 = 0.9653
weight2 = 0.9229
total = weight1 + weight2
weight1 = weight1/total
weight2 = weight2/total

In [36]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 8936
})

In [37]:
import concurrent.futures
label = []
results = []

def process_image(image):
    combined_results = {}
    result1 = pipe1(image)
    result2 = pipe2(image)

    for item in result1:
        combined_results[item['label']] = combined_results.get(item['label'], 0) + item['score'] * weight1
    for item in result2:
        combined_results[item['label']] = combined_results.get(item['label'], 0) + item['score'] * weight2

    # Find the label with the maximum combined score
    max_label = max(combined_results, key=combined_results.get)
    return max_label

for i in range(0, len(ds), 32):
    batch = ds[i:i+32]
    label.extend(batch["label"])
    print(i)
    # Process each image in the batch concurrently
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results.extend(executor.map(process_image, batch["image"]))


print(label)
print(results)
        


0
32
64


KeyboardInterrupt: 

In [ ]:
label = []
results = []

for i in range(0, len(ds), 8):
    batch = ds[i:i+8]
    label.extend(batch["label"])
    for image in batch["image"]:
        combined_results = {}
        result1 = pipe1(image)
        result2 = pipe2(image)

        for item in result1:
            combined_results[item['label']] = item['score'] * weight1
        for item in result2:
            if item['label'] in combined_results:
                combined_results[item['label']] += item['score'] * weight2
            else:
                combined_results[item['label']] = item['score'] * weight2
        results.append(next(iter(combined_results)))

print(label)
print(results)
        